In [1]:
import os
import sys
from pathlib import Path
from collections import defaultdict

from tqdm import tqdm
import pandas as pd
import yaml

cwd = Path.cwd()
if cwd.name == 'notebooks':
    os.chdir(cwd.parent)
sys.path.append('./scripts')

from cr_racing_results import RaceResults

In [71]:
def get_team_results(race_results: pd.DataFrame) -> pd.DataFrame:
    team_gr = race_results.loc[race_results.status == 'Q'].groupby('team')
    n_riders = team_gr.name.count()
    results = team_gr.result_time.nsmallest(3).groupby(level=0).sum()
    n_riders.name = 'n_riders'
    team_results = pd.DataFrame([n_riders, results]).T
    team_results.loc[team_results.n_riders > 3, 'n_riders'] = 3
    team_results = team_results.sort_values(['n_riders', 'result_time'], ascending=[False, True]).reset_index()
    team_results['place'] = team_results.index + 1
    return team_results.loc[:, ['team', 'result_time', 'place']]

In [79]:
racer_teams = defaultdict(list)
for race_folder in os.listdir('data/races'):
    race = RaceResults.load(os.path.join('data/races', race_folder))
    for cluster, result in race.group.items():
        racer_teams[cluster].append(result.loc[result.status == 'Q', ['name', 'year_of_birth', 'team']])

In [92]:
df = pd.concat(racer_teams['A']).drop_duplicates()

In [95]:
df.groupby(['name', 'year_of_birth']).team.count().sort_values()

name                year_of_birth
Музычкин Антон      1994             0
Обыдённов Алексей   1976             0
Овсянников Никита   1993             0
Овчинников Серафим  1990             0
Одинцов Иван        2003             0
                                    ..
Першуков Михаил     1972             3
Демёхин Андрей      1972             3
Беликов Юрий        1987             3
Шамбер Владислав    1974             3
Жак Михаил          1979             3
Name: team, Length: 187, dtype: int64

In [98]:
df.loc[(df.name == 'Демёхин Андрей')]

,name,year_of_birth,team
28,Демёхин Андрей,1972,Кластер А. Магадан А
33,Демёхин Андрей,1972,Магадан. Кластер А
49,Демёхин Андрей,1972,Магадан


In [78]:
os.listdir('data/races')

['Верея', 'Дубна', 'Садовое кольцо']

In [74]:
get_team_results(race.group['B'])

,team,result_time,place
0,Магадан,24109.409,1
1,HBFS Plan B,24113.704,2
2,Legion Cycling Club,24114.826,3
3,Volga Union MCA 2,24115.591,4
4,SlowFlow,24116.491,5
5,RCA-2,24118.163,6
6,Volga Union MCA 3,17318.875,7


In [69]:
team_results = team_results.sort_values(['n_riders', 'result_time'], ascending=[False, True]).reset_index()
team_results['place'] = team_results.index + 1

In [70]:
team_results

,team,n_riders,result_time,place
0,Кластер B. RCA 3,3.0,31198.534,1
1,Кластер B. HBFS Plan B,3.0,31485.331,2
2,Кластер B. SERPANTIN,2.0,20795.532,3
3,Кластер B. Магадан В,2.0,21103.011,4
4,Кластер B. Volga Union MCA 2,2.0,22595.272,5


In [68]:
tr

,team,n_riders,result_time,place
0,Кластер B. RCA 3,3.0,31198.534,1
1,Кластер B. HBFS Plan B,3.0,31485.331,2
2,Кластер B. SERPANTIN,2.0,20795.532,3
3,Кластер B. Магадан В,2.0,21103.011,4
4,Кластер B. Volga Union MCA 2,2.0,22595.272,5
